In [1]:
from causaldag.structure_learning import gspo
import causaldag as cd
import random
import numpy as np
from causaldag.utils.ci_tests import MemoizedCI_Tester, gauss_ci_test, gauss_ci_suffstat

## Setup graph, samples, and sufficient statistic

In [2]:
nnodes = 5
nodes = set(range(nnodes))
nlatent = 2
exp_nbrs = 2
nsamples = 1000

Sample a random Erdos-Renyi DAG with random weights. Then, take $n$ samples and forget the first `nlatent` values.

In [3]:
random.seed(2861298674)
np.random.seed(9863)

dag = cd.rand.directed_erdos(nnodes+nlatent, exp_nbrs/(nnodes+nlatent/2))
gdag = cd.rand.rand_weights(dag)
all_samples = gdag.sample(nsamples)
observed_samples = all_samples[:, nlatent:]

Compute the sufficient statistics for the conditional independence test you are using. Pass those sufficient statistics and the conditional independence testing function to a `CI_Tester`. In this case, we use partial-correlation based CI testing.

In [4]:
suffstat = gauss_ci_suffstat(observed_samples)
ci_tester = MemoizedCI_Tester(gauss_ci_test, suffstat)

## Estimate the MAG and compare to the true MAG

Compute the ground-truth MAG.

In [5]:
mag = dag.marginal_mag(set(range(nlatent)), relabel='default')

Estimate a MAG using GSPo.

In [6]:
est_mag = gspo(nodes, ci_tester)

In [7]:
est_mag.shd_skeleton(mag)

3